<a href="https://colab.research.google.com/github/HeatherDriver/IU-Model-Engineering/blob/main/04_Model_and_KFold_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install ordered_set
! pip install wandb
! pip install "pandas<2.0.0"
! pip install -U kaleido

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.6/289.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 76.2 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
google-colab 1.0.0 requires pandas==2.0.3, but you have pandas 1.5.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 7.5 MB/s eta 0:00:

In [2]:
import pandas as pd
import numpy as np
import warnings
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from sklearn import preprocessing as pp
import pickle
import os
from sklearn.decomposition import PCA
from sklearn.multioutput import MultiOutputRegressor
from itertools import product
import wandb
from sklearn.model_selection import KFold, ParameterGrid, GridSearchCV, LeaveOneOut, cross_val_score
from sklearn.linear_model import SGDRegressor, LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
import xgboost as xgb
from ordered_set import OrderedSet
from sklearn.metrics import mean_squared_error

In [3]:
try:
    from xgboost.core import _LIB

    cuda_available = True
except ImportError:
    cuda_available = False

if cuda_available:
    print("CUDA is available for XGBoost.")
else:
    print("CUDA is not available for XGBoost.")

CUDA is available for XGBoost.


In [4]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/01_Data/03_Modelling'

pkl_path = '/content/drive/MyDrive/01_Data/03_Modelling/'
images_folder = '/content/drive/MyDrive/02_Docs'

Mounted at /content/drive
/content/drive/MyDrive/01_Data/03_Modelling


In [5]:
# Display all columns, rows and import the data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None

warnings.filterwarnings('ignore')

In [6]:
# Function for importing pickle file
def pkl_reader(filename):
  with open(filename, 'rb') as file:
    loaded_object = pickle.load(file)
  return loaded_object

In [7]:
# Function to start Weights and Biases to log model runs with various hyperparameter combinations

def start_wandb(proj_name, proj_notes, config_dict):
  ! wandb login db6beed0e5e6cc44c1537a34fd548aa868475178

  USER_NAME = "heather-rink"
  # start a new wandb run to track
  run = wandb.init(
      # set the wandb project where this run will be logged
      project=proj_name,
      notes=proj_notes,
      config=config_dict
      )

In [8]:
# Reading in file
X_pca = pkl_reader('X_test_for_kfold.pkl')
y_pca = pkl_reader('y_for_pca.pkl')
y_additional = pkl_reader('y_additional_for_pca.pkl')
groundtime_correlation_dict = pkl_reader('Groundtime_corr_coeff.pkl') # Maps to co_eff, intercept
duration_correlation_dict = pkl_reader('Duration_corr_coeff.pkl')

# Converting dataframes to numpy arrays
X_pca_matrix = [X.values for X in X_pca]
y_pca_matrix = [y.values for y in y_pca]
y_additional_matrix = [y.values for y in y_additional] # maps to 'sched_duration' and 'Sched_Groundtime'

In [9]:
y_names = ['actual_duration', 'dep_delay', 'Act_Groundtime_imputed']
y_additional = ['sched_duration', 'Sched_Groundtime_imputed']

In [10]:
# Defining the number of folds for k-fold cross validation
folds = range(2, 30, 5)

## Developing a Baseline Model

In [11]:
# Function for RMSE calculation
def calcs_rmse(y_actual, y_hat):
  rmse = np.sqrt(np.mean((y_actual - y_hat)**2, axis=0))
  return rmse

# Baseline model uses the zero rule, which serves as a better baseline model than a random prediction setup.
# This will use the mean of the observed values in the train dataset.
def creates_baseline_model_performance():
  mydict = dict()

  for y_name, X, y in zip(y_names, X_pca_matrix, y_pca_matrix):
    _mydict = dict()

    for k_folds in folds:
      kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)
      for train_index, test_index in kf.split(X):
        X_train, X_valid = X[train_index], X[test_index]
        y_train, y_valid = y[train_index], y[test_index]

      y_valid_mean = np.mean(y_valid, axis=0)
      repeated_y_valid_mean = np.tile(y_valid_mean, (y_valid.shape[0], 1))

      assert repeated_y_valid_mean.shape == y_valid.shape, 'Error'

      rmse_valid = calcs_rmse(y_valid, repeated_y_valid_mean)
      rmse_valid = np.mean(rmse_valid)
      _mydict[k_folds] = rmse_valid
      mydict.update({y_name : _mydict})
  return mydict

In [12]:
# Extending baseline model to the 2 calculated variables (scheduled duration and scheduled ground time)
def extends_baseline_model_performance(baseline_dict):
  for y_name, X, y in zip(y_additional, X_pca_matrix, y_additional_matrix):
    if y_name == 'sched_duration':
      X = X_pca_matrix[0]
    if y_name == 'Sched_Groundtime_imputed':
      X = X_pca_matrix[2]
    _mydict = dict()

    for k_folds in folds:
      kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)
      for train_index, test_index in kf.split(X):
        X_train, X_valid = X[train_index], X[test_index]
        y_train, y_valid = y[train_index], y[test_index]
      y_valid_mean = np.mean(y_valid, axis=0)
      repeated_y_valid_mean = np.tile(y_valid_mean, (y_valid.shape[0], 1))

      assert repeated_y_valid_mean.shape == y_valid.shape, 'Error'

      rmse_valid = calcs_rmse(y_valid, repeated_y_valid_mean)
      rmse_valid = np.mean(rmse_valid)
      _mydict[k_folds] = rmse_valid
      baseline_dict.update({y_name : _mydict})
  return baseline_dict

In [13]:
# Results to a dataframe
baseline_dict = creates_baseline_model_performance()
baseline_dict = extends_baseline_model_performance(baseline_dict)

y_names.extend(y_additional)

mylist = []
for name in y_names:
  my_df = pd.DataFrame.from_dict(baseline_dict[name], orient='index').reset_index(drop=False)
  my_df.columns = ['n_splits' ,'Baseline']
  my_df['Predicted_Variable'] = name
  my_df['Average_Baseline'] = my_df['Baseline'].mean()
  mylist.append(my_df)

baseline_df = pd.concat(mylist).reset_index(drop=True)

In [14]:
# Save the df to pkl
with open('baselines.pkl', 'wb') as f:
    pickle.dump(baseline_df, f)

In [15]:
# Visualising the Baseline
fig = px.bar(baseline_df, x='n_splits', y='Baseline', color='Predicted_Variable', facet_col='Predicted_Variable', color_discrete_sequence=px.colors.sequential.Viridis, title='Baseline Model performance per k-fold split, per Predicted Variable')

facet_columns = baseline_df['Predicted_Variable'].unique()
for col in facet_columns:
    mean_baseline = baseline_df[baseline_df['Predicted_Variable'] == col]['Average_Baseline'].mean()
    rounded_mean = np.round(mean_baseline, 2)
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    fig.add_hline(y=mean_baseline, line_dash="dot", line_color="red", annotation_text=f"Mean Baseline: {rounded_mean}", row=1, col=facet_columns.tolist().index(col) + 1)
fig.update_layout(title=None, showlegend=False, width=1600, height=600)
fig.write_image(images_folder + "/baseline_per_kfold_split.png")
fig.show()

## Testing K-fold splits across different Models & different hyperparameter combinations

In [16]:
notes_for_run = 'Test'
regressor_dict = {'SGDRegressor' : 'SG Descent', 'SVRegressor' : 'SV Regression', 'KNeighborsRegressor' : 'K Nearest Neighbours Regression', 'XGBRegressor' : 'XG Boost', 'RandomForestRegressor' : 'Random Forest'}

In [17]:
X_pca_matrix.extend([X_pca_matrix[0]])
X_pca_matrix.extend([X_pca_matrix[2]])

y_pca_matrix.extend(y_additional_matrix)

In [18]:
# Function to calculate scheduled ground time, and scheduled duration from the relevant regression coefficients for _0, _1, _2 etc
def _calcs_pred_groundtime_sched_duration(key, input_matrix):
  # Empty array
  corr_vector = np.empty((1, 11, 2))

  # Iterate through the dictionary to create vector
  for i in range(0, 11):
    if 'sched_duration' in key:
      corr_vector[0, i] = groundtime_correlation_dict[('quant_transf_Act_Groundtime_imputed_' + str(i), 'quant_transf_Sched_Groundtime_imputed_' + str(i))]
    else:
      corr_vector[0, i] = duration_correlation_dict[('quant_transf_actual_duration_' + str(i), 'quant_transf_sched_duration_' + str(i))]

  corr_vector_reshaped = corr_vector[:, :input_matrix.shape[1], :]
  result = input_matrix * corr_vector_reshaped[:,:,0] + corr_vector_reshaped[:,:,1]
  return result

In [19]:
# Function to return validation set for each k_fold applied
def _returns_train_validation_matrix(folds):
    mydict = dict()
    for k_folds in folds:
      kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)
      for y_name, X, y in zip(y_names, X_pca_matrix, y_pca_matrix):
        tup = (k_folds, y_name)
        for train_index, test_index in kf.split(X):
          X_train, X_valid = X[train_index], X[test_index]
          y_train, y_valid = y[train_index], y[test_index]
          mydict.update({tup : [X_train, y_train, X_valid, y_valid]})
    return mydict

# Functon to return a dictionary with RMSE for each variable
def _creates_model_results(regressor, train_validation_matrix, param_grid):
    mydict = dict()
    for line in param_grid:
      param_line = line.copy()

      for key, value in train_validation_matrix.items():
        X_train, y_train, X_valid, y_valid = value

        # Fitting model to the 3 predicted variables
        if key not in y_additional:
          if regressor == 'SGDRegressor':
            model = SGDRegressor(**line)
          if regressor == 'SVRegressor':
            model = svm.SVR(**line)
          if regressor == 'KNeighborsRegressor':
            model = KNeighborsRegressor(**line)
          regr = MultiOutputRegressor(model).fit(X_train, y_train)

          # Predictions
          y_hat_valid = regr.predict(X_valid)

        # Calculating correlated variables via coefficients derived from previous analysis
        else:
          y_hat_valid = _calcs_pred_groundtime_sched_duration(key, y_hat_valid)

        # Prediction evaluation
        rmse_valid  = calcs_rmse(y_valid, y_hat_valid)
        rmse_valid_mean  =  np.mean(rmse_valid, axis=0)
        rmse_valid_max  =  np.max(rmse_valid, axis=0)
        rmse_valid_min =  np.min(rmse_valid, axis=0)

        mylist = []
        param_line.update({'n_splits' : key[0], 'y_name' : key[1], 'model' : regressor})
        new_param_line = tuple(param_line.items())
        mydict[new_param_line] = [rmse_valid_mean, rmse_valid_min, rmse_valid_max]
    return mydict

# Function to create dataframe with above results
def _creates_results_df(model_results, regressor):
  mylist = []
  for key, value in model_results.items():
    _df = pd.DataFrame(key)
    _df = _df.T
    _df.columns = _df.iloc[0]
    _df = _df[1:]
    _df = _df.reset_index(drop=True)
    rmse_df = pd.DataFrame(data={'rmse': [value[0]]})
    min_df = pd.DataFrame(data={'rmse_min': [value[1]]})
    max_df = pd.DataFrame(data={'rmse_max': [value[2]]})

    df = pd.concat([_df, rmse_df, min_df, max_df], axis=1)
    last_column_name = df['y_name'].values[0]
    df.columns = [*df.columns[:-3], last_column_name, last_column_name + '_min', last_column_name + '_max']
    df.drop(columns=['y_name'], inplace=True)
    mylist.append(df)

  df = pd.concat(mylist, axis=0).reset_index(drop=True)
  if regressor == 'SVRegressor':
    grouped_df = df.groupby(by=df.columns.tolist()[0:3], as_index=False).first()
  else:
    grouped_df = df.groupby(by=df.columns.tolist()[0:5], as_index=False).first()
  return grouped_df

# Function returning the dataframe from above
def runs_model(folds, param_grid, regressor):
  train_validation_matrix = _returns_train_validation_matrix(folds)
  model_results = _creates_model_results(regressor, train_validation_matrix, param_grid)
  results_df = _creates_results_df(model_results, regressor)
  return results_df

# Due to Google Colab sessions timing out while executing - this updates the parameter grid according to what hyperparameter combinations
# have not yet been run so that these can run next time.
def model_param_grid_updater(param_grid, results_df):
    my_dict = {}
    mylist = []

    # Iterate over each dictionary in param_grid to create df
    for item in param_grid:
      for key, value in item.items():
        if key in my_dict:
          my_dict[key].append(value)
        else:
          my_dict[key] = [value]

    param_grid_df = pd.DataFrame.from_dict(my_dict)
    columns_names = param_grid_df.columns.to_list()

    if results_df.empty == True:
      zeros_row_series = pd.Series(0, index=columns_names)
      results_df = pd.DataFrame([zeros_row_series])

    results_df_subset = results_df[param_grid_df.columns.to_list()]

    for i, row in param_grid_df.iterrows():
      mydict = {}
      conditions = []

      for column in param_grid_df.columns:
        conditions.append(f"(results_df_subset['{column}'] == row.{column})")
      subset = results_df_subset.loc[eval(' & '.join(conditions))]

      if subset.empty:
        mydict.update(row)
        mylist.append(mydict)
    return mylist

# Logs results within Weights and Biases application for model and hyperparameter combination assessment
def logs_results_wandb(df, project_name):
    if param_grid:
      # Log interim results in W&B
      for i in range(0, df.shape[0]):
        row = df.iloc[i]
        wandb_dict = row.to_dict()

        run = start_wandb(proj_name=project_name, proj_notes='Test', config_dict=wandb_dict)
        wandb.config = wandb_dict
        filtered_dict = {key: wandb_dict[key] for key in wandb_dict if key in y_names}
        wandb.log(filtered_dict)
        wandb.finish()

### K-Nearest neighbours regression

In [20]:
# Trains and evaluates a K Nearest neighbours model using the training and validation dataset with a grid of hyperparameters.

file_name = pkl_path + 'KNeighborsRegressor_dataframe.pkl'

# Check if the pickle file exists
if os.path.exists(file_name):
  knr_df = pd.read_pickle(file_name)
else:
  knr_df = pd.DataFrame()

KNR = {
            "n_neighbors": [30, 40, 50, 60, 70],
            "weights": ['uniform'],
            "algorithm": ['auto'],
            "leaf_size": [15, 30]
        }
_param_grid = list(ParameterGrid(KNR))

param_grid = model_param_grid_updater(_param_grid, knr_df)

if param_grid:
  _knr_df = runs_model(folds, param_grid, 'KNeighborsRegressor')

  knr_df = pd.concat([knr_df, _knr_df], axis=0)
  knr_df.drop_duplicates(inplace=True)
  knr_df.to_pickle(file_name)
  logs_results_wandb(knr_df, 'K Nearest Neighbours Regression')

### Stochastic Gradient Descent

In [21]:
# Trains and evaluates a Stochastic Gradient Descent model using the training and validation dataset with a grid of hyperparameters.

file_name = pkl_path + 'SGD_dataframe.pkl'

# Check if the pickle file exists
if os.path.exists(file_name):
  sgd_df = pd.read_pickle(file_name)
else:
  sgd_df = pd.DataFrame()

SGDescent = {
            "penalty": ['l1','l2', 'elasticnet'],
            "loss" : ['squared_error', 'huber'],
            "alpha": [0.0001, 0.001, 0.01, 0.1], #multiplier for regularization term
            "max_iter" : [100, 1000], #max number of passes for training
        }
_param_grid = list(ParameterGrid(SGDescent))

param_grid = model_param_grid_updater(_param_grid, sgd_df)

if param_grid:
  _sgd_df = runs_model(folds, param_grid, 'SGDRegressor')

  sgd_df = pd.concat([sgd_df, _sgd_df], axis=0)
  sgd_df.drop_duplicates(inplace=True)
  sgd_df.to_pickle(file_name)
  logs_results_wandb(sgd_df, 'SG Descent')

### Support Vector Regression

In [22]:
# Trains and evaluates a Support Vector model using the training and validation dataset with a grid of hyperparameters.

file_name = pkl_path + 'SupportVector_dataframe.pkl'

# Check if the pickle file exists
if os.path.exists(file_name):
  svr_df = pd.read_pickle(file_name)
else:
  svr_df = pd.DataFrame()

SVR = {
            "kernel": ['rbf', 'linear'],
            "C": [0.01, 0.1, 0.15, 0.2, 0.5, 1.0], #regularization
        }
_param_grid = list(ParameterGrid(SVR))

param_grid = model_param_grid_updater(_param_grid, svr_df)

if param_grid:
  _svr_df = runs_model(folds, param_grid, 'SVRegressor')

  svr_df = pd.concat([svr_df, _svr_df], axis=0)
  svr_df.drop_duplicates(inplace=True)
  svr_df.to_pickle(file_name)
  logs_results_wandb(svr_df, 'SV Regression')

### XG Boost

In [23]:
# Trains and evaluates a XG Boost model using the training and validation dataset with a grid of hyperparameters.

# Due to XG Boost's configuration, need to rework previous functions to fit in with change.
def _returns_train_validation_dmatrix(folds):
    mydict = dict()
    for k_folds in folds:
        kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)
        for y_name, X, y in zip(y_names, X_pca_matrix, y_pca_matrix):
            tup = (k_folds, y_name)
            for train_index, test_index in kf.split(X):
                X_train, X_valid = X[train_index], X[test_index]
                y_train, y_valid = y[train_index], y[test_index]
                dtrain = xgb.DMatrix(X_train, label=y_train)
                dval = xgb.DMatrix(X_valid, label=y_valid)
                mydict.update({tup : [dtrain, dval, y_valid]})
    return mydict

# Function returning a dictionary of the RMSE for the model's predictions and the feature importances for each parameter combination.
def _runs_xgboost(dtrain, dval, y_valid, param_grid, keys):
    mydict = dict()
    for param_line in param_grid:
      if keys not in y_additional:
        model = xgb.train(param_line, dtrain, num_boost_round=param_line['num_boost_round'], evals=[(dtrain, 'train'), (dval, 'validation')])
        y_hat_valid = model.predict(dval)
      else:
        y_hat_valid = _calcs_pred_groundtime_sched_duration(keys, y_hat_valid)
      rmse_valid = np.sqrt(mean_squared_error(y_valid, y_hat_valid))
      mytup = tuple((key, value) for key, value in param_line.items())
      feature_importance = model.get_score(importance_type='weight')
      mydict[mytup] = rmse_valid, feature_importance
    return mydict

# Returns dictionary with results
def _creates_xgboost_results(dmatrix_dict, param_grid):
    mydict = dict()
    for keys, values in dmatrix_dict.items():
        dtrain, dval, y_valid = values
        xg_dict = _runs_xgboost(dtrain, dval, y_valid, param_grid, keys)
        for xg_keys, xg_values in xg_dict.items():
            param_line = dict(xg_keys)
            rmse_valid, feature_importance = xg_values
            param_line.update({'n_splits' : keys[0], 'y_name' : keys[1], 'random_state' : 42, 'shuffle' : True, 'model' : 'XG Boost'})
            new_param_line = tuple(param_line.items())
            mydict[new_param_line] = [rmse_valid, feature_importance]
    return mydict

# Returns dataframe of results
def creates_xgboost_df(param_grid):
    dmatrix_dict = _returns_train_validation_dmatrix(folds)
    mydict = _creates_xgboost_results(dmatrix_dict, param_grid)

    mylist = []
    for key, value in mydict.items():
      _df = pd.DataFrame(key)
      _df = _df.T
      _df.columns = _df.iloc[0]
      _df = _df[1:]
      _df = _df.reset_index(drop=True)
      rmse_df = pd.DataFrame(data={'rmse': [value[0]]})

      df = pd.concat([_df, rmse_df], axis=1)
      last_column_name = df.iloc[0, 9]
      df.columns = [*df.columns[:-1], last_column_name]
      df.drop(columns=['y_name'], inplace=True)
      mylist.append(df)

    df = pd.concat(mylist, axis=0).reset_index(drop=True)
    grouped_df = df.groupby(by=df.columns.tolist()[0:12], as_index=False).first()
    return grouped_df

# Updates parameter grid, in case Colab session times out so that entire process does not need to be re-run from scratch
def xgb_param_grid_updater(param_grid, xgb_df):
    my_dict = {}
    mylist = []

    # Iterate over each dictionary in param_grid to create df
    for item in param_grid:
      for key, value in item.items():
        if key in my_dict:
          my_dict[key].append(value)
        else:
          my_dict[key] = [value]

    param_grid_df = pd.DataFrame.from_dict(my_dict)
    columns_names = param_grid_df.columns.to_list()

    if xgb_df.empty == True:
      zeros_row_series = pd.Series(0, index=columns_names)
      results_df = pd.DataFrame([zeros_row_series])
      xgb_df_subset = results_df[param_grid_df.columns.to_list()]
    else:
      xgb_df_subset = xgb_df.copy()

    for i, row in param_grid_df.iterrows():
      mydict = dict()
      subset = xgb_df_subset.loc[
            (xgb_df_subset['alpha'] == row.alpha) &
            (xgb_df_subset['early_stopping_rounds'] == row.early_stopping_rounds) &
            (xgb_df_subset['eval_metric'] == row.eval_metric) &
            (xgb_df_subset['lambda'] == row['lambda']) &
            (xgb_df_subset['learning_rate'] == row.learning_rate) &
            (xgb_df_subset['max_depth'] == row.max_depth) &
            (xgb_df_subset['num_boost_round'] == row.num_boost_round) &
            (xgb_df_subset['objective'] == row.objective)
      ]
      if subset.empty is True:
        mydict.update({'alpha' : row.alpha, 'early_stopping_rounds' : row.early_stopping_rounds, 'eval_metric' : row.eval_metric, 'lambda' : row['lambda'], 'learning_rate' : row.learning_rate,
                       'max_depth' : row.max_depth, 'num_boost_round' : row.num_boost_round, 'objective' : row.objective})
        mylist.append(mydict)
    return mylist

In [24]:
file_name = pkl_path + 'XGBoost_dataframe.pkl'

# Check if the pickle file exists
if os.path.exists(file_name):
  xgb_df = pd.read_pickle(file_name)
else:
  xgb_df = pd.DataFrame()

# alpha is the L1 regularizer
# lambda is the L2 regularizer

XGB = {
    'objective': ['reg:squarederror'],  # Regression objective
    'eval_metric': ['rmse'],  # Evaluation metric
    'num_boost_round': [120],
    'learning_rate' : [0.1, 0.3],
    'max_depth': [2, 3],
    'early_stopping_rounds': [50],
    'lambda': [0.01, 0.1, 1, 10, 100, 300],  # L2 regularization
    'alpha': [0.001, 0.01, 0.1],  # L1 regularization
    # 'tree_method': ['cuda'],  # Enable GPU acceleration = 'cpu'
}

param_grid = list(ParameterGrid(XGB))

param_grid = xgb_param_grid_updater(param_grid, xgb_df)

if param_grid:
  _xgb_df = creates_xgboost_df(param_grid)
  xgb_df = pd.concat([xgb_df, _xgb_df], axis=0).reset_index(drop=True)

xgb_df.drop_duplicates(inplace=True)

xgb_df.to_pickle(file_name)

In [25]:
logs_results_wandb(xgb_df, 'XG Boost')

### Random Forest

In [26]:
# Trains and evaluates a Random Forest model using the training and validation dataset with a grid of hyperparameters.

# Function returning a dictionary of the RMSE for the model's predictions and the feature importances for each parameter combination.
def _creates_random_forest_results(train_validation_matrix, param_grid):
    mydict = dict()
    for line in param_grid:
      param_line = line.copy()
      model = RandomForestRegressor(**line)

      for key, value in train_validation_matrix.items():
        X_train, y_train, X_valid, y_valid = value
        if key not in y_additional:
          regr = MultiOutputRegressor(model).fit(X_train, y_train)
          y_hat_valid = regr.predict(X_valid)
        else:
          y_hat_valid = _calcs_pred_groundtime_sched_duration(key, y_hat_valid)
        feature_importance, mylist = [], []

        for estimator in regr.estimators_:
          _importance = pd.Series(estimator.feature_importances_).sort_values(ascending=False)
          _importance = _importance[_importance > 0]
          mylist.append(_importance)

        # Get the average importance across all estimators
        combined_importances = pd.concat(mylist, axis=1).mean(axis=1)
        combined_importances.sort_values(ascending=False)
        feature_importance.append(combined_importances)

        rmse_valid = np.sqrt(mean_squared_error(y_valid, y_hat_valid))
        param_line.update({'n_splits' : key[0], 'y_name' : key[1], 'model' : 'Random Forest'})
        new_param_line = tuple(param_line.items())
        mydict[new_param_line] = [rmse_valid, feature_importance]
    return mydict

# Returns results within a dataframe
def _creates_random_forest_df(random_forest_results):
    if not random_forest_results:
        # Return an empty DataFrame with a consistent structure
        return pd.DataFrame()

    mylist = []
    for key, value in random_forest_results.items():
        _df = pd.DataFrame(key)
        _df = _df.T
        _df.columns = _df.iloc[0]
        _df = _df[1:]
        _df = _df.reset_index(drop=True)
        rmse_df = pd.DataFrame(data={'rmse': [value[0]]})

        df = pd.concat([_df, rmse_df], axis=1)
        last_column_name = df.iloc[0, 6]
        df.columns = [*df.columns[:-1], last_column_name]
        df.drop(columns=['y_name'], inplace=True)
        mylist.append(df)
    # Concatenate all DataFrames in mylist
    if mylist:
        concatenated_df = pd.concat(mylist, axis=0).reset_index(drop=True)
        grouped_df = concatenated_df.groupby(by=concatenated_df.columns.tolist()[0:6], as_index=False).first()
    else:
        # Return an empty DataFrame if there's no valid data to process
        grouped_df = pd.DataFrame()
    return grouped_df

# Creates dictionary with feature importances
def _creates_feature_importances_dict(random_forest_results):
  mydict = dict()
  for key, value in random_forest_results.items():
    mydict[key] = value[1]
  return mydict

# Runs the process
def random_forest_model(param_grid):
  train_validation_matrix = _returns_train_validation_matrix(folds)
  random_forest_results = _creates_random_forest_results(train_validation_matrix, param_grid)
  random_forest_df = _creates_random_forest_df(random_forest_results)
  random_forest_importances = _creates_feature_importances_dict(random_forest_results)
  return random_forest_df, random_forest_importances

# Updates the hyperparameter grid so that if the session times out it does not need to be re-run from scratch.
def rf_param_grid_updater(param_grid, random_forest_df):
    my_dict = {}
    mylist = []

    # Iterate over each dictionary in param_grid to create df
    for item in param_grid:
      for key, value in item.items():
        if key in my_dict:
          my_dict[key].append(value)
        else:
          my_dict[key] = [value]

    param_grid_df = pd.DataFrame.from_dict(my_dict)
    columns_names = param_grid_df.columns.to_list()

    if random_forest_df.empty == True:
      zeros_row_series = pd.Series(0, index=columns_names)
      results_df = pd.DataFrame([zeros_row_series])
      random_forest_df_subset = results_df[param_grid_df.columns.to_list()]
    else:
      random_forest_df_subset = random_forest_df.copy()

    for i, row in param_grid_df.iterrows():
      mydict = dict()
      subset = random_forest_df_subset.loc[
            (random_forest_df_subset['criterion'] == row.criterion) &
            (random_forest_df_subset['max_depth'] == row.max_depth) &
            (random_forest_df_subset['min_samples_leaf'] == row.min_samples_leaf) &
            (random_forest_df_subset['n_estimators'] == row.n_estimators) &
            (random_forest_df_subset['random_state'] == row.random_state)
      ]

      if subset.empty is True:
        mydict.update({'criterion' : row.criterion, 'max_depth' : row.max_depth, 'min_samples_leaf' : row.min_samples_leaf, 'n_estimators' : row.n_estimators, 'random_state' : row.random_state})
        mylist.append(mydict)
    return mylist

In [27]:
# Running above model

file_name = pkl_path + 'RandomForest_dataframe.pkl'

# Check if the pickle file exists
if os.path.exists(file_name):
  rf_df = pd.read_pickle(file_name)
else:
  rf_df = pd.DataFrame()

Random_Forest = {
            "n_estimators": [100, 200], # Number of trees
            "max_depth": [20], #Splits
            "criterion" : ["squared_error"],
            "min_samples_leaf" : [0.12, 0.05, 0.2, 0.3], # Each leaf contains at least x % of the data used in training
            "random_state" : [1]
        }
_param_grid = list(ParameterGrid(Random_Forest))

param_grid = rf_param_grid_updater(_param_grid, rf_df)

_rf_df, rf_importances = random_forest_model(param_grid)

rf_df = pd.concat([rf_df, _rf_df], axis=0)
rf_df.drop_duplicates(inplace=True)

rf_df.to_pickle(file_name)

In [28]:
logs_results_wandb(rf_df, 'Random Forest')

## Comparing the performance of models on the data

In [29]:
# Concatenating all results within one df
all_results = pd.concat([rf_df, xgb_df, knr_df, sgd_df, svr_df], axis=0)

In [30]:
# Re-ordering such that the best RMSE per model is selected to top_models df
_mylist = []

for y_name in y_names:
  avg = sum(baseline_dict[y_name].values()) / len(baseline_dict[y_name].values())
  mylist = []

  for model_name in all_results['model'].unique():
    subset = all_results.loc[all_results['model'] == model_name, [y_name, 'model']].copy().sort_values(by=[y_name]).head(1)
    subset['Avg_baseline'] = avg
    mylist.append(subset)

  mydf = pd.concat(mylist, axis=0)
  mydf.columns = ['RMSE', 'model', 'avg_baseline']
  mydf['predicted_variable'] = y_name

  _mylist.append(mydf)

top_models = pd.concat(_mylist, axis=0).reset_index(drop=True)

In [31]:
top_models

,RMSE,model,avg_baseline,predicted_variable
0,0.448329,Random Forest,0.641396,actual_duration
1,0.314522,XG Boost,0.641396,actual_duration
2,0.366267,KNeighborsRegressor,0.641396,actual_duration
3,0.219893,SGDRegressor,0.641396,actual_duration
4,0.215280,SVRegressor,0.641396,actual_duration
5,0.547993,Random Forest,0.610180,dep_delay
6,0.446125,XG Boost,0.610180,dep_delay
7,0.439066,KNeighborsRegressor,0.610180,dep_delay
8,0.313409,SGDRegressor,0.610180,dep_delay
9,0.301737,SVRegressor,0.610180,dep_delay


In [32]:
# Visualising the data

fig = px.bar(top_models, x='model', y='RMSE', facet_col='predicted_variable', color='model', color_discrete_sequence=px.colors.sequential.Viridis, title='Best RMSE per model, per Predicted Variable versus the Mean Baseline')
facet_columns = top_models['predicted_variable'].unique()
for col in facet_columns:
    mean_baseline = top_models[top_models['predicted_variable'] == col]['avg_baseline'].mean()
    rounded_mean = np.round(mean_baseline, 2)
    fig.add_hline(y=mean_baseline, line_dash="dot", line_color="red", annotation_text=f"Baseline: {rounded_mean}", row=1, col=facet_columns.tolist().index(col) + 1)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_layout(title=None, showlegend=False, width=1600)
fig.update_xaxes(title=None)

fig.write_image(images_folder + "/Best_RMSE_per_predicted_variable.png")
fig.show()

## Results of best k-fold cross validation per model

In [33]:
# Saving optimised hyperparameter combinations to a dictionary
model_params = {}
model_params['sched_duration'] = svm.SVR(C=0.1, kernel='linear')
model_params['actual_duration'] = svm.SVR(C=0.1, kernel='linear')
model_params['dep_delay'] = svm.SVR(C=0.15, kernel='linear')
model_params['Act_Groundtime_imputed'] = svm.SVR(C=0.2, kernel='linear')
model_params['Sched_Groundtime_imputed'] = svm.SVR(kernel='linear', C=0.5)

In [34]:
# Checking which number of folds best suits for predicion

filename = 'LeaveOneOut_results.pkl'
if os.path.exists(filename):
  loo_dict = pkl_reader(filename)
else:
  # Initialize Leave-One-Out Cross-Validator
  loo = LeaveOneOut()

  loo_dict = dict()
  mylist = []

  # Leave-One-Out Cross-Validation
  for y_name, X, y in zip(y_names, X_pca_matrix, y_pca_matrix):
    # Initialise relevant model
    model = model_params[y_name]

    rmse_list = []

    for train_index, test_index in loo.split(X):
      # Split data into training and test sets
      X_train, X_test = X[train_index], X[test_index]
      y_train, y_test = y[train_index], y[test_index]

      # Fit the model on training data
      regr = MultiOutputRegressor(model).fit(X_train, y_train)

      # Make predictions on test data
      y_hat = regr.predict(X_test)

      # RMSE for this iteration
      rmse = np.sqrt(mean_squared_error(y_test, y_hat))
      rmse_list.append(rmse)
      # Mean RMSE across all iterations
      mean_rmse = np.mean(rmse_list, axis=0)
      mylist.append(mean_rmse)

    mean_rmse = np.mean(rmse_list)
    loo_dict[y_name] = mean_rmse

  with open(filename, 'wb') as f:
    pickle.dump(loo_dict, f)

In [35]:
# Gets the baseline from the dictionary so that it can be added to the graphs
def returns_baseline(var_name):
  baseline = []
  for val in to_graph['n_splits'].to_list():
    baseline.append(baseline_dict[var_name][val])
  return baseline

# Function to plot model performance against the Baseline and the Ideal
def plots_model_versus_ideal_and_baseline(y_name, graph_file_name=False):
    min_name = y_name + '_min'
    max_name = y_name + '_max'

    # Offset for separating traces
    offset = 0.8
    fig = go.Figure()

    # Baseline data
    fig.add_trace(go.Scatter(
        x=to_graph['n_splits'] + offset,
        y=to_graph['baseline'],
        mode='markers',
        name='Baseline',
        marker=dict(color=px.colors.sequential.Viridis[0], size=12)
    ))

    # Model data
    fig.add_trace(go.Scatter(
        x=to_graph['n_splits'] - offset,
        y=to_graph[y_name],
        mode='markers',
        name=y_name,
        marker=dict(color=px.colors.sequential.Viridis[5], size=12),
        error_y=dict(
            type='data',
            symmetric=False,
            array=to_graph[max_name],  # Max error values
            arrayminus=to_graph[min_name]  # Min error values
        )
    ))

    fig.add_hline(y=loo_dict[y_name], annotation_text="Leave One Out Ideal RMSE: " + str(np.round(loo_dict[y_name], 4)), line_dash="dot")
    fig.add_hline(y=to_graph['avg_baseline'].unique()[0], annotation_text="Avg Baseline: " + str(np.round(to_graph['avg_baseline'].unique()[0], 2)), line_dash="dot")

    fig.update_xaxes(title_text='n_splits')
    fig.update_yaxes(title_text=y_name)
    fig.update_layout(title='Comparison of Baseline versus Ideal and Model performance for folds for ' + y_name)

    if graph_file_name:
      fig.update_layout(title=None, showlegend=False, width=1600, height=600)
      fig.write_image(images_folder + "/" + graph_file_name)
    fig.show()

In [36]:
# Visualising the above for scheduled duration

to_graph = all_results.loc[(all_results['C'] == 0.1) & (all_results['kernel'] == 'linear')][['sched_duration', 'sched_duration_min', 'sched_duration_max', 'n_splits']]
to_graph['max'] = to_graph['sched_duration_max'] - to_graph['sched_duration']
to_graph['min'] = to_graph['sched_duration'] - to_graph['sched_duration_min']
to_graph['baseline'] = returns_baseline('sched_duration')
to_graph['avg_baseline'] = to_graph['baseline'].mean()

plots_model_versus_ideal_and_baseline('sched_duration')

In [37]:
# Based on the above analysis, using k=27 gets the model performing close to the ideal, while sufficiently outperforming the baseline
k_fold_dict = dict()
k_fold_dict.update({'sched_duration' : 27})

In [38]:
# Visualising for actual duration

to_graph = all_results.loc[(all_results['C'] == 0.1) & (all_results['kernel'] == 'linear')][['actual_duration', 'actual_duration_min', 'actual_duration_max', 'n_splits']]
to_graph['max'] = to_graph['actual_duration_max'] - to_graph['actual_duration']
to_graph['min'] = to_graph['actual_duration'] - to_graph['actual_duration_min']
to_graph['baseline'] = returns_baseline('actual_duration')
to_graph['avg_baseline'] = to_graph['baseline'].mean()

plots_model_versus_ideal_and_baseline('actual_duration', 'actual_duration_folds.png')

In [39]:
# Based on the above graphs, using k=27 will allow the closest approximation to the ideal
k_fold_dict.update({'actual_duration' : 27})

In [40]:
# Visualising for departure delay
to_graph = all_results.loc[(all_results['C'] == 0.15) & (all_results['kernel'] == 'linear')][['dep_delay', 'dep_delay_min', 'dep_delay_max', 'n_splits']]
to_graph['max'] = to_graph['dep_delay_max'] - to_graph['dep_delay']
to_graph['min'] = to_graph['dep_delay'] - to_graph['dep_delay_min']
to_graph['baseline'] = returns_baseline('dep_delay')
to_graph['avg_baseline'] = to_graph['baseline'].mean()

plots_model_versus_ideal_and_baseline('dep_delay')

In [41]:
# Based on the above graphs, using k=27 will allow the closest approximation to the ideal
k_fold_dict.update({'dep_delay' : 27})

In [42]:
# Visualising for actual ground time
to_graph = all_results.loc[(all_results['C'] == 0.2) & (all_results['kernel'] == 'linear')][['Act_Groundtime_imputed', 'Act_Groundtime_imputed_min', 'Act_Groundtime_imputed_max', 'n_splits']]
to_graph['max'] = to_graph['Act_Groundtime_imputed_max'] - to_graph['Act_Groundtime_imputed']
to_graph['min'] = to_graph['Act_Groundtime_imputed'] - to_graph['Act_Groundtime_imputed_min']
to_graph['baseline'] = returns_baseline('Act_Groundtime_imputed')
to_graph['avg_baseline'] = to_graph['baseline'].mean()

plots_model_versus_ideal_and_baseline('Act_Groundtime_imputed')

In [43]:
# Based on the above graphs, using k=27 will allow the closest approximation to the ideal
k_fold_dict.update({'Act_Groundtime_imputed' : 27})

In [44]:
# Visualising for scheduled ground time
to_graph = all_results.loc[(all_results['kernel'] == 'linear') & (all_results['C'] == 0.5)][['Sched_Groundtime_imputed', 'Sched_Groundtime_imputed_min', 'Sched_Groundtime_imputed_max', 'n_splits']]
to_graph['max'] = to_graph['Sched_Groundtime_imputed_max'] - to_graph['Sched_Groundtime_imputed']
to_graph['min'] = to_graph['Sched_Groundtime_imputed'] - to_graph['Sched_Groundtime_imputed_min']
to_graph['baseline'] = returns_baseline('Sched_Groundtime_imputed')
to_graph['avg_baseline'] = to_graph['baseline'].mean()

plots_model_versus_ideal_and_baseline('Sched_Groundtime_imputed')

In [45]:
# Based on the above graphs, using k=27 will allow the closest approximation to the ideal
k_fold_dict.update({'Sched_Groundtime_imputed' : 27})

In [46]:
# Save the dictionaries to pkl files for implementation with the tets dataset.
with open('models_dict.pkl', 'wb') as f:
    pickle.dump(model_params, f)

with open('kfold_dict.pkl', 'wb') as f:
    pickle.dump(k_fold_dict, f)

print(model_params)
print(k_fold_dict)

{'sched_duration': SVR(C=0.1, kernel='linear'), 'actual_duration': SVR(C=0.1, kernel='linear'), 'dep_delay': SVR(C=0.15, kernel='linear'), 'Act_Groundtime_imputed': SVR(C=0.2, kernel='linear'), 'Sched_Groundtime_imputed': SVR(C=0.5, kernel='linear')}
{'sched_duration': 27, 'actual_duration': 27, 'dep_delay': 27, 'Act_Groundtime_imputed': 27, 'Sched_Groundtime_imputed': 27}
